# 1.Librerías

In [ ]:
%pip install lightgbm==4.4.0
%pip install optuna==3.6.1

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

# 2.Datos

In [ ]:
#seteo carpeta de trabajo y archivo
dataset_path = '/home/ingceciliaconde/buckets/b1/datasets'
modelos_path = '/home/ingceciliaconde/buckets/b1/datasets/modelos'
db_path = '/home/ingceciliaconde/buckets/b1/datasets/db'
exp_path ='/home/ingceciliaconde/buckets/b1/exp'
dataset_file = 'competencia_02_fe_v1.csv.gz'


In [ ]:
#path de trabajo local para pruebas
# #seteo carpeta de trabajo y archivo
# dataset_path = 'C:/Users/mconde/Documents/DMEYF/Competencia_01'
# db_path = 'C:/Users/mconde/Documents/DMEYF/Competencia_01'
# modelos_path = 'C:/Users/mconde/Documents/DMEYF/Competencia_01'
# dataset_file = 'competencia_01.csv'


In [ ]:
print(f"{dataset_path}/{dataset_file}")



In [4]:
# Lectura de datos.
data= pd.read_csv(f"{dataset_path}/{dataset_file}")


# Parametros

In [5]:
#semillas y valores de ganancia
semillas=[100183,200003,300017,700001,800011]
ganancia_acierto = 273000
costo_estimulo = 7000

In [6]:
#meses
mes_train_all = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
                 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
                 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]

mes_train_6 = [202101, 202102, 202103, 202104, 202105, 202106]
mes_train_12 = [202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106]
mes_test = 202108

In [ ]:
mes_train=mes_train_all #si uso 6 meses, cambiar a mes_train_6 y si uso all, cambiar a mes_train_all

## Darle valor a las clases

In [8]:
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

En particular, sumaremos la clase BAJA+1, que es estructuralmente muy similar a BAJA+2, para aumentar los casos positivos. Luego, compararemos los resultados obtenidos con los de la clase con la que hemos estado trabajando hasta ahora.

In [9]:
data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0) # solo baja +2
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1) #ambas bajas

In [11]:
#filtrar por baja + 2 y mostrar clase ternaria clase binaria
data[data['clase_ternaria'] == 'BAJA+2'][['clase_ternaria', 'clase_binaria1', 'clase_binaria2']].head()


,clase_ternaria,clase_binaria1,clase_binaria2
1047,BAJA+2,1,1
1651,BAJA+2,1,1
1870,BAJA+2,1,1
1920,BAJA+2,1,1
2039,BAJA+2,1,1


# 3.Separa Test y Train

In [12]:
train_data = data[data['foto_mes'].isin(mes_train)] # train_data = data[data['foto_mes'] == mes_train]
#test_data = data[data['foto_mes'] == mes_test]
test_data = data[data['foto_mes'] == 202106] #solo para hacer el script, en la competencia se usará 202108

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_train_binaria1 = train_data['clase_binaria1'] #SOLO BAJA+2
y_train_binaria2 = train_data['clase_binaria2'] #TODAS LAS BAJAS
w_train = train_data['clase_peso']

X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

# 4.Funcion de Ganancia

In [13]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Parámetros del modelos.
params = {
    'objective': 'binary',
    'metric': 'gan_eval',
    'boosting_type': 'gbdt',
    'max_bin': 31,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.7,
    'verbose': 0
}

# Corre el modelo

In [36]:
#8. Volvemos a leer el modelo.
#model_lgb = lgb.Booster(model_file= modelos_path + 'lgbm_{}_{}.txt'.format(cantidad_meses_train,ventana))

In [37]:
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()

ModuleNotFoundError: No module named 'kaggle'

In [ ]:
import os
import re

# Definimos el patrón de nombre que esperamos para los archivos de modelos
pattern = re.compile(r'^lgb_v+\.txt$') #modificar segun nomrbre de archivo

# Obtén la lista de archivos de modelos en el directorio que coinciden con el patrón
model_files = [f for f in os.listdir(dataset_path) if pattern.match(f)]
print(model_files)

# Ensamble de probabilidades

In [ ]:
import pandas as pd
import numpy as np
import os
import time

# Inicialización de DataFrame para almacenar todas las predicciones
df_predicciones = pd.DataFrame()

# Bucle sobre cada archivo de modelo para cargar y predecir
for model_file in model_files:
    model_path = os.path.join(dataset_path, model_file)
    model_lgb = lgb.Booster(model_file=model_path)
    
    # Copia X_test para evitar añadir múltiples veces la columna 'Probabilidad'
    X_test_copy = X_test.copy()
    
    # Predecir con el modelo cargado y almacenar las probabilidades
    predicciones = model_lgb.predict(X_test_copy)
    df_predicciones[model_file] = predicciones



In [ ]:
# Asegurarse de que ambos DataFrames tienen el mismo índice
if not df_predicciones.index.equals(X_test.index):
    print("Los índices no son iguales. Realineando...")
    df_predicciones.index = X_test.index


In [ ]:
# Verificar que las columnas esperadas estén presentes y que no haya NaNs
print(df_predicciones.head())
print("¿Hay valores NaN en las predicciones?:", df_predicciones.isna().any().any())

In [ ]:
# Calcular la media de las predicciones de todos los modelos
X_test['Probabilidad'] = df_predicciones.mean(axis=1)
tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

In [ ]:
# Ver el DataFrame final para confirmar que los datos son correctos
print(X_test_copy[['numero_de_cliente', 'Probabilidad']].head())


In [ ]:

import time
# Asegúrate de que num_subida_kaggle esté inicializado
# Parámetros de tu competencia y subidas
competencia = 'dm-ey-f-2024-segunda'
num_subida_kaggle = 1  # o cargar el último valor si estás continuando subidas previas

# Cortes candidatos y subidas
cortes = range(9000, 14000, 500)
for envios in cortes:
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[['numero_de_cliente', 'Predicted']].reset_index(drop=True)
    
    nombre_archivo = f"C2_CC_30_fev1_ensamblepromedio{num_subida_kaggle}.csv"
    ruta_archivo = f"{exp_path}/{nombre_archivo}"
    resultados.to_csv(ruta_archivo, index=False)
    
    mensaje = f'Archivo {nombre_archivo}. Ensamble, DF {ventana}, fev1, punto_corte: {envios}.'
    
    # Envío a Kaggle
    if num_subida_kaggle <= 15:
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
    else:
        print("Esperamos 30 segundos...")
        time.sleep(30)
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
    
    num_subida_kaggle += 1
